# SABR Model

In this notebook we analyse Normal implied volatility smile interpolation using **S**tochastic **A**lpha **B**eta **R**ho model.

The model is formulated in terms of a forward rate $S(t)$. For our applications, the forward rate is a forward swap rate.

The forward rate $S(t)$ is assumed to be a martingale in a suitable pricing measure. For swaption pricing, the martingale pricing measure is the *annuity measure*.

Model dynamics are
$$
\begin{aligned}
  dS(t)            &=\hat{\alpha}(t)\cdot S(t)^{\beta}\cdot dW(t), \\
  d\hat{\alpha}(t) &=\nu\cdot\hat{\alpha}(t)\cdot dZ(t), \\
  \hat{\alpha}(0)  &=\alpha, \\
  dW(t)\cdot dZ(t) &=\rho\cdot dt.
\end{aligned}
$$

Model parameters have the following impact in implied volatilities:

  - $\alpha$ controls the overall level of implied volatilities.

  - $\beta$ and $\rho$ change the slope (or skew) of implied volatilities.

  - $\nu$ controls the curvature (or smile) of implied volatilities.


In [ ]:
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import numpy as np

from src.sabr_model import SabrModel

## Static Smile Modelling

We set up a few example SABR models where we successively *switch on* individual parameters.

In [ ]:
S0 = 0.05 # forward rate 5%
T = 5.0 # 5y expiry

# SabrModel( S(t), T, alpha, beta, nu, rho )
model1 = SabrModel(S0, T, 0.0100, 0.0001, 0.0001, 0.0)
model2 = SabrModel(S0, T, 0.0450, 0.5000, 0.0001, 0.0)
model3 = SabrModel(S0, T, 0.0405, 0.5000, 0.5000, 0.0)
model4 = SabrModel(S0, T, 0.0420, 0.5000, 0.5000, 0.7)

For our example models we can plot Normal implied volatilities.

In [ ]:
strikes = np.linspace(1.0e-4, 0.10, 100)
#
vols1 = np.array([model1.normal_volatility(strike) for strike in strikes])
vols2 = np.array([model2.normal_volatility(strike) for strike in strikes])
vols3 = np.array([model3.normal_volatility(strike) for strike in strikes])
vols4 = np.array([model4.normal_volatility(strike) for strike in strikes])

plt.figure(figsize=(8,5))
plt.plot(strikes, vols1, 'b-', label='Normal model')
plt.plot(strikes, vols2, 'r-', label='CEV model')
plt.plot(strikes, vols3, 'g-', label='CEV+SV model')
plt.plot(strikes, vols4, 'y-', label='CEV+SV+Corr model')
plt.legend()
plt.ylim((0.00,0.02))
plt.xlabel('strike')
plt.ylabel('Normal implied volatility')
plt.title(r'SABR smiles, $T=5$, $S(0)=5\%$')

The example smiles confirm the general impact of the model parameters.

In addition we can also calculate and plot model-implied densities of the terminal distribution of the swap rate.

In [ ]:
rates = strikes
#
dens1 = np.array([model1.density(S) for S in rates])
dens2 = np.array([model2.density(S) for S in rates])
dens3 = np.array([model3.density(S) for S in rates])
dens4 = np.array([model4.density(S) for S in rates])

plt.figure(figsize=(8,5))
plt.plot(rates, dens1, 'b-', label='Normal model')
plt.plot(rates, dens2, 'r-', label='CEV model')
plt.plot(rates, dens3, 'g-', label='CEV+SV model')
plt.plot(rates, dens4, 'y-', label='CEV+SV+Corr model')
plt.ylim((-20, 40))
plt.legend()
plt.xlabel('rate $S(T)$')
plt.ylabel('implied density')
plt.title(r'SABR densities, $T=5$, $S(0)=5\%$')

## Approximation Accuracy

The Normal implied volatility formula in SABR model is an approximation. A natural question is how accurate the approximation is.

In order to assess approximation accuracy we use a Monte Carlo simulation. With the Monte Carlo simulation we simulate paths of the forward rate $S(t)$ (and stochastic volatility $\hat \alpha(t)$).

Based on a set of simulated paths we approximate expectations in call and put option pricing formulas by averages of sampled payoffs. Once, we have the option prices we can calculate Normal model-implied volatilities.

We start with simulating model paths.

In [ ]:
from src.monte_carlo_simulation import MonteCarloSimulation

times  = np.array([k*0.01 for k in range(501)])

times = np.linspace(0.0, T, 501)
n_paths = 2**13
# times = np.linspace(0.0, T, 1251)
# n_paths = 2**16

sim1 = MonteCarloSimulation(model1,times,n_paths, showProgress=True)
sim2 = MonteCarloSimulation(model2,times,n_paths, showProgress=True)
sim3 = MonteCarloSimulation(model3,times,n_paths, showProgress=True)
sim4 = MonteCarloSimulation(model4,times,n_paths, showProgress=True)

Next, we calculate implied volatilities from simulated paths.

In [ ]:
from src.helpers import bachelier_implied_vol

def implied_volatility_monte_carlo(sim, T, strikes):
    idx = np.searchsorted(sim.times, T)
    assert sim.times[idx] == T
    S_0 = sim.X[0,0,0]
    S_T = sim.X[idx,0,:]
    S_T += (S_0 - S_T.mean()) # we incorporate an adjuster to numerically ensure put-call-parity
    S_T = np.reshape(S_T, (-1,1))
    K = np.reshape(strikes, (1,-1))
    #
    V_T = np.maximum((2*(K>S_0)-1) * (S_T - K), 0.0)
    E_T_T = np.mean(V_T, axis=0)
    vols = np.array([
         bachelier_implied_vol(P_, K_, S_0, T, 2*(K_>S_0)-1) 
         for P_, K_ in zip(E_T_T, strikes)
         ])
    return vols


In [ ]:
ref_strikes = np.linspace(0.01, 0.10, 10)

vols1_1y_mc = implied_volatility_monte_carlo(sim1, T=1.0, strikes=ref_strikes)
vols2_1y_mc = implied_volatility_monte_carlo(sim2, T=1.0, strikes=ref_strikes)
vols3_1y_mc = implied_volatility_monte_carlo(sim3, T=1.0, strikes=ref_strikes)
vols4_1y_mc = implied_volatility_monte_carlo(sim4, T=1.0, strikes=ref_strikes)

vols1_5y_mc = implied_volatility_monte_carlo(sim1, T=5.0, strikes=ref_strikes)
vols2_5y_mc = implied_volatility_monte_carlo(sim2, T=5.0, strikes=ref_strikes)
vols3_5y_mc = implied_volatility_monte_carlo(sim3, T=5.0, strikes=ref_strikes)
vols4_5y_mc = implied_volatility_monte_carlo(sim4, T=5.0, strikes=ref_strikes)


For comparison we also calculate the corresponding SABR vols.

In [ ]:
def implied_volatility_model(model, T, strikes):
     """Wrap vol calculation for variable T"""
     m = SabrModel(model.forward, T, model.alpha, model.beta, model.nu, model.rho)
     return np.array([m.normal_volatility(strike) for strike in strikes])

vols1_1y_md = implied_volatility_model(model1, T=1.0, strikes=strikes)
vols2_1y_md = implied_volatility_model(model2, T=1.0, strikes=strikes)
vols3_1y_md = implied_volatility_model(model3, T=1.0, strikes=strikes)
vols4_1y_md = implied_volatility_model(model4, T=1.0, strikes=strikes)

vols1_5y_md = implied_volatility_model(model1, T=5.0, strikes=strikes)
vols2_5y_md = implied_volatility_model(model2, T=5.0, strikes=strikes)
vols3_5y_md = implied_volatility_model(model3, T=5.0, strikes=strikes)
vols4_5y_md = implied_volatility_model(model4, T=5.0, strikes=strikes)


And finally we plot and compare the results.

In [ ]:
# 1y plot
plt.figure(figsize=(8,5))
plt.plot(strikes,     vols1_1y_md, 'b-', label='Normal model')
plt.plot(ref_strikes, vols1_1y_mc, 'b*')
plt.plot(strikes,     vols2_1y_md, 'r-', label='CEV model')
plt.plot(ref_strikes, vols2_1y_mc, 'r*')
plt.plot(strikes,     vols3_1y_md, 'g-', label='CEV+SV model')
plt.plot(ref_strikes, vols3_1y_mc, 'g*')
plt.plot(strikes,     vols4_1y_md, 'y-', label='CEV+SV+Corr model')
plt.plot(ref_strikes, vols4_1y_mc, 'y*')
plt.legend()
plt.ylim((0.005,0.025))
plt.xlabel('strike')
plt.ylabel('Normal implied volatility')
plt.title(r'SABR smiles, $T=1$, $S(0)=5\%$')
# 5y plot
plt.figure(figsize=(8,5))
plt.plot(strikes,     vols1_5y_md, 'b-', label='Normal model')
plt.plot(ref_strikes, vols1_5y_mc, 'b*')
plt.plot(strikes,     vols2_5y_md, 'r-', label='CEV model')
plt.plot(ref_strikes, vols2_5y_mc, 'r*')
plt.plot(strikes,     vols3_5y_md, 'g-', label='CEV+SV model')
plt.plot(ref_strikes, vols3_5y_mc, 'g*')
plt.plot(strikes,     vols4_5y_md, 'y-', label='CEV+SV+Corr model')
plt.plot(ref_strikes, vols4_5y_mc, 'y*')
plt.legend()
plt.ylim((0.005,0.025))
plt.xlabel('strike')
plt.ylabel('Normal implied volatility')
plt.title(r'SABR smiles, $T=5$, $S(0)=5\%$')

We find that Monte Carlo implied volatilities match SABR approximation volatilites well for smaller expiries. However, for larger expiries and stochastic volatility we do see differences between Monte Carlo implied volatilities and SABR approximation volatilites.

## Smile Dynamics

In this section we want to analyse how the implied volatility smile changes is the underlying forward swap rate increases or decreases. This behaviour is important in particular for Delta risk calculation.

We set up two SABR models with low and higher elasticity parameter $\beta$. The correlation parameter $\rho$ is adjusted such that both models show a similar skew and smile around at-the-money.

We also plot the smile of a pure CEV model (without stochastic volatility) to verify that CEV cannot model volatility curvature.

In [ ]:
# SabrModel( S(t), T, alpha, beta, nu, rho )
model1 = SabrModel(0.05,5.0,0.0420, 0.1000,0.5000,0.3 )
model2 = SabrModel(0.05,5.0,0.0420, 0.7000,0.5000,0.0 )
model3 = SabrModel(0.05,5.0,0.0420, 0.9000,0.0001,0.0 )
# ATM calibration
print(model1.calibrate_atm(0.01), model2.calibrate_atm(0.01), model3.calibrate_atm(0.01))

vols1 = [model1.normal_volatility(strike) for strike in strikes]
vols2 = [model2.normal_volatility(strike) for strike in strikes]
vols3 = [model3.normal_volatility(strike) for strike in strikes]

plt.figure(figsize=(8,5))
plt.plot(strikes,vols1, 'b-', label='beta=0.1,nu=0.5,rho=0.3')
plt.plot(strikes,vols2, 'r-', label='beta=0.7,nu=0.5,rho=0.0')
plt.plot(strikes,vols3, 'g-', label='beta=0.9,nu=0.0,rho=0.0')
plt.legend()
plt.xlabel('strike')
plt.ylabel('Normal volatility')
plt.xlim((0.0, 0.10))
plt.ylim((0.005, 0.020))

Now, we slide the forward rate $S(0)$ from $2\%$ to $8\%$. Then we plot the resulting new smiles and highlight at-the-money volatilities.

In [ ]:
S_ = [ 0.020, 0.035, 0.050, 0.065, 0.080 ]
vols1_ = []
vols2_ = []
vols3_ = []
backBone1_ = []
backBone2_ = []
backBone3_ = []
for S in S_:
    model1.forward = S
    model2.forward = S
    model3.forward = S
    vols1_.append([model1.normal_volatility(strike) for strike in strikes])
    vols2_.append([model2.normal_volatility(strike) for strike in strikes])
    vols3_.append([model3.normal_volatility(strike) for strike in strikes])
    backBone1_.append(model1.normal_volatility(S))
    backBone2_.append(model2.normal_volatility(S))
    backBone3_.append(model3.normal_volatility(S))

# beta = 0.1
plt.figure(figsize=(8,5))
for k in range(len(S_)):
    plt.plot(strikes,vols1_[k], 'b:', label='S='+str(S_[k]))
plt.plot(S_,backBone1_, 'bo-')
plt.legend()
plt.xlabel('swap rate')
plt.ylabel('Normal volatility')
plt.xlim((0.0, 0.10))
plt.ylim((0.005, 0.020))
# plt.savefig('SABRSmileSVBeta01.png', dpi=150)

# beta = 0.7
plt.figure(figsize=(8,5))
for k in range(len(S_)):
    plt.plot(strikes,vols2_[k], 'r:', label='S='+str(S_[k]))
plt.plot(S_,backBone2_, 'ro-')
plt.legend()
plt.xlabel('swap rate')
plt.ylabel('Normal volatility')
plt.xlim((0.0, 0.10))
plt.ylim((0.005, 0.020))
# plt.savefig('SABRSmileSVBeta07.png', dpi=150)

# CEV
plt.figure(figsize=(8,5))
for k in range(len(S_)):
    plt.plot(strikes,vols3_[k], 'g:', label='S='+str(S_[k]))
plt.plot(S_,backBone3_, 'go-')
plt.legend()
plt.xlabel('swap rate')
plt.ylabel('Normal volatility')
plt.xlim((0.0, 0.10))
plt.ylim((0.003, 0.020))
# plt.savefig('SABRSmileLVBeta09.png', dpi=150)


Low $\beta$ yields horizontal shift of the volatility smile, high $\beta$ moves the smile upwards. If we can observe the volatility behaviour e.g. from historical data then we can use the $\beta$ parameter to match our model to these observations.

For CEV model we also see that volatilities rise if forward rate increases. However, the volatility shape yields appearance that the smile moves to the left if forward moves right. This observation is sometimes considered contradictory to market observations.

## Shifted SABR Model

The code in this section requires the implementation of the Shifted SABR model, see exercises.

We need to reset some inputs in order to correspond to our market data example from Vanilla models.

In [ ]:
T = 5.0
S0 = 0.0050  # 0.5%
strikes = np.linspace(-0.01, 0.02, 301)

Via manual guessing we find SABR model parameters that match our market quotes from the basic Vanilla model analysis. 

In [ ]:
from src.shifted_sabr_model import ShiftedSabrModel

shifted_model = ShiftedSabrModel(S0, T, 0.0538, 0.7, 0.239, -0.021, 0.05)

Now, we can repeat the analysis from the basic Vanilla models and add implied volatilities from our Shifted SABR model.

In [ ]:
from src.helpers import black

relative_strikes = np.array([ -150, -100, -50, -25, 0, 25, 50, 100, 150 ]) * 1.0e-4
smile_quotes = np.array([ -3.97, -2.93, -1.73, -0.94, 0.0, 1.11, 2.39, 5.42, 9.00 ]) * 1.0e-4
atm_vol_quote = 72.02 * 1.0e-4
#
def implied_vol_from_shifted_lognormal_model(T, strike, forward, sigma_sln, shift_lambda):
    callOrPut = 1 if strike > S0 else -1
    fwd_price = black(strike + shift_lambda, S0 + shift_lambda, sigma_sln, T, callOrPut)
    return bachelier_implied_vol(fwd_price, strike, forward, T, callOrPut)
#
implied_vol_n = atm_vol_quote * np.ones(len(strikes))

sigma_sln = 0.085
shift_lambda = 0.08
implied_vol_sln = np.array([ implied_vol_from_shifted_lognormal_model(5.0, K, S0, sigma_sln, shift_lambda) for K in strikes ])

implied_vol_shifted_sabr = np.array([
    shifted_model.normal_volatility(K) for K in strikes
])

plt.figure(figsize=(8,5))
plt.plot(S0+relative_strikes, atm_vol_quote+smile_quotes, 'm*', label='market quotes')
plt.plot(strikes, implied_vol_n, 'b', label='Normal model')
plt.plot(strikes, implied_vol_sln, 'g', label='Shifted Log-normal model')
plt.plot(strikes, implied_vol_shifted_sabr, 'r', label='Shifted SABR model')
plt.ylim(0.0060, 0.0085)
plt.xlabel('strike')
plt.ylabel('Normal implied volatility')
plt.legend()

As a result we see that the Shifted SABR model allows for a very good fit to our market quotes.